In [ ]:
# define matplotlibplotting backend
# %matplotlib -l shows all available backends
%matplotlib qt

In [ ]:
import os, sys
sys.path.append(os.path.join(os.path.dirname(os.path.abspath(os.getcwd())), "lib/python"))

import numpy as np

from picopic.plot_builder import PlotBuilder
from picopic.h5_reader import H5Reader


In [ ]:
##  configuration options
data_path = '/home/cosmonaut/dev/pic/picopic'

radius=0.02
longitude=0.2
show_grid=False
use_cache=False
autoselect = True
ylim_r=None
ylim_z=None

if os.path.isfile(os.path.join(data_path, "data.h5")):
    reader = H5Reader(path = data_path, use_cache=use_cache, verbose=True)
else:
    raise EnvironmentError("There is no corresponding data/metadata files in the path " + data_path + ". Can not continue.")

x_axis_label = r'$\mathit{t (s)}$'
y_r_axis_label = r'$\mathit{E_r (\frac{V}{m})}$'
y_z_axis_label = r'$\mathit{E_z (\frac{V}{m})}$'

# time_range=[reader.meta.time[0], reader.meta.time[1]]
time_range=[0, 1.5e-11]

e_r_plot_name = r'$\mathbf{Electrical\enspace Field\enspace Radial\enspace Component}\enspace(E_r)$'
e_z_plot_name = r'$\mathbf{Electrical\enspace Field\enspace Longitudal\enspace Component}\enspace(E_z)$'

In [ ]:
# get data
start_frame = None # cfg.get_frame_number_by_timestamp(time_range[0])
end_frame = None # 460 # cfg.get_frame_number_by_timestamp(time_range[1])
row_number = reader.meta.get_row_by_radius(radius)
col_number = reader.meta.get_col_by_longitude(longitude)

data_r = []
data_z = []

for probe in reader.meta.probes:
    if (probe.shape == 'dot') and (probe.size[0] == row_number) and (probe.size[1] == col_number):
        start_frame = reader.meta.get_frame_number_by_timestamp(time_range[0], probe.schedule)
        end_frame = reader.meta.get_frame_number_by_timestamp(time_range[1], probe.schedule)
        if probe.component == 'E/r': data_r = reader.dot_range('E/r', row_number, col_number, start_frame, end_frame)
        if probe.component == 'E/z': data_z = reader.dot_range('E/z', row_number, col_number, start_frame, end_frame)
            
if (len(data_r) == 0 or len(data_z) == 0) and autoselect:
    for probe in reader.meta.probes:
        if (probe.shape == 'rec') and (probe.size[0] <= row_number) and (probe.size[1] <= col_number) and (probe.size[2] >= row_number) and (probe.size[3] >= col_number):
            start_frame = reader.meta.get_frame_number_by_timestamp(time_range[0], probe.schedule)
            end_frame = reader.meta.get_frame_number_by_timestamp(time_range[1], probe.schedule)
            shape = [probe.size[0], probe.size[1], probe.size[2], probe.size[3]]
            # for piece in range(start_frame, end_frame):
            if probe.component == 'E/r':
                data_r = reader.dot_rec_range('E/r', row_number, col_number, shape, start_frame, end_frame)
            if probe.component == 'E/z':
                data_z = reader.dot_rec_range('E/z', row_number, col_number, shape, start_frame, end_frame)

In [ ]:
data_timeline = np.linspace(time_range[0], time_range[1], len(data_r))

# define plot builder
plot = PlotBuilder(0, 0, # let the system detects sizes automatically
                   fig_color=reader.meta.figure_color, 
                   fig_width=reader.meta.figure_width,
                   fig_height=reader.meta.figure_height, 
                   fig_dpi=reader.meta.figure_dpi,
                   font_family=reader.meta.figure_font_family,
                   font_name=reader.meta.figure_font_name,
                   font_size=reader.meta.figure_font_size,
                   tickbox=True, grid=show_grid, is_invert_y_axe=False,
                   aspect='auto', guess_number_ticks=20,
                   # number_x_ticks=10, number_y_ticks=10
                   x_ticklabel_end=1e-9, y_ticklabel_end=1e-9
                  )

# add subplots
plot_r = plot.add_subplot_cartesian_2d(e_r_plot_name, 121, x_axe_label=x_axis_label, y_axe_label=y_r_axis_label)
plot_z = plot.add_subplot_cartesian_2d(e_z_plot_name, 122, x_axe_label=x_axis_label, y_axe_label=y_z_axis_label)

# set y-limits
if ylim_r is not None:
    plot_r.set_ylim(ylim_r)

if ylim_z is not None:
    plot_z.set_ylim(ylim_z)

In [ ]:
# add data
plot_r.plot(data_timeline, data_r)
plot_z.plot(data_timeline, data_z)

plot.show()